In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from sklearn import metrics
import seaborn as sns

Using TensorFlow backend.


In [2]:
drug_data = pd.read_csv("C:/Users/Rui Wu/Documents/Programming C Drive/Final Project/cleaned_drug_new.csv")
drug_data.shape

(1885, 20)

In [3]:
drug_data_y = drug_data['Illicit Drugs']
drug_data_x = drug_data.drop(['Illicit Drugs', 'ID'], axis = 1)
# drug_data_x = drug_data_x.astype('category')

drug_dummies = pd.get_dummies(drug_data_x[drug_data_x.columns])
drug_dummies.head()

,Age,Education,Nscore,Escore,Oscore,Ascore,Cscore,Impulsive,SS,Gender_Female,...,Legal High_CL4,Legal High_CL5,Legal High_CL6,Nicotine_CL0,Nicotine_CL1,Nicotine_CL2,Nicotine_CL3,Nicotine_CL4,Nicotine_CL5,Nicotine_CL6
0,0.49788,-0.05921,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,-0.21712,-1.18084,1,...,0,0,0,0,0,1,0,0,0,0
1,-0.07854,1.98437,-0.67825,1.93886,1.43533,0.76096,-0.14277,-0.71126,-0.21575,0,...,0,0,0,0,0,0,0,1,0,0
2,0.49788,-0.05921,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,-1.37983,0.40148,0,...,0,0,0,1,0,0,0,0,0,0
3,-0.95197,1.16365,-0.14882,-0.80615,-0.01928,0.59042,0.58489,-1.37983,-1.18084,1,...,0,0,0,0,0,1,0,0,0,0
4,0.49788,1.98437,0.73545,-1.63340,-0.45174,-0.30172,1.30612,-0.21712,-0.21575,1,...,0,0,0,0,0,1,0,0,0,0


In [4]:
kf = KFold(n_splits=4, shuffle=True, random_state=0)
i = 0
scores = []
total_cm = [[0,0],[0,0]]
for train_index, test_index in kf.split(drug_dummies):
    i = i + 1
    print("Iteration " + str(i))
    x_train, x_test = drug_dummies.iloc[train_index], drug_dummies.iloc[test_index]
    y_train, y_test = drug_data_y.iloc[train_index], drug_data_y.iloc[test_index]
    
    model = Sequential()
    model.add(Dense(128, input_dim=len(drug_dummies.columns), activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='tanh'))
    model.add(Dense(8, activation='tanh'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.fit(x_train, y_train, epochs=100, batch_size=10, verbose=0)
    
    predictions = model.predict_classes(x_test)
    correct = 0
    
    for j in range(len(predictions)):
        if (predictions[j] == y_test.iloc[j]):
            correct = correct + 1
    score = correct / len(predictions)
    scores.append(score)
    print(score)
    cm = metrics.confusion_matrix(y_test, predictions)
    print(cm)

    for a in range(len(cm)):
        for b in range(len(cm)):
            total_cm[a][b] += cm[a][b]

print("True Negatives Rate: " + str(total_cm[0][0] / 1154))
print("True Positives Rate: " + str(total_cm[1][1] / 731))
print("False Negatives Rate: " + str(total_cm[1][0] / 731))
print("False Positives Rate: " + str(total_cm[0][1] / 1154))

scores_mean = sum(scores) / len(scores)
scores_std = np.std(np.array(scores))

print("Overall Accuracy: %0.4f (+/- %0.4f)" % (scores_mean, scores_std * 2))

Iteration 1
0.7754237288135594
[[236  59]
 [ 47 130]]
Iteration 2
0.7303609341825902
[[214  51]
 [ 76 130]]
Iteration 3
0.7558386411889597
[[233  65]
 [ 50 123]]
Iteration 4
0.7600849256900213
[[238  58]
 [ 55 120]]
True Negatives Rate: 0.7980935875216638
True Positives Rate: 0.6880984952120383
False Negatives Rate: 0.3119015047879617
False Positives Rate: 0.2019064124783362
Overall Accuracy: 0.7554 (+/- 0.0324)


In [5]:
# model = Sequential()
# model.add(Dense(32, input_dim=67, activation='relu'))
# model.add(Dense(16, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))

In [6]:
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
# model.fit(x_train, y_train, epochs=50, batch_size=10)

In [8]:
# predictions = model.predict_classes(x_test)

In [9]:
# cm = metrics.confusion_matrix(y_test, predictions)
# print(cm)